In [1]:
import os
import findspark

from alkansya.utils import get_configurations
from alkansya.feature_engineering import spark_query as query

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

import pandas as pd
from datetime import datetime

os.environ["ENV"] = "DEV"
findspark.init()

cfg = get_configurations()

PATH_TO_BRONZE = cfg["path_to_bronze"]
PATH_TO_SILVER = cfg["path_to_silver"]
PATH_TO_GOLD = cfg["path_to_gold"]

spark = SparkSession.builder.master("local[*]").getOrCreate()

23/12/30 11:53:45 WARN Utils: Your hostname, Nigels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.254.124 instead (on interface en0)
23/12/30 11:53:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 11:53:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dfs = spark.read.parquet(f"{PATH_TO_GOLD}/resolution_60min")

dfs_temp = dfs.select(
    "*",
    query.simple_moving_average(window_size_days=1),
    query.exponential_moving_average(window_size_days=1),
    query.volatility(window_size_days=1),
    query.simple_moving_average(window_size_days=3),
    query.exponential_moving_average(window_size_days=3),
    query.volatility(window_size_days=3),
    # *query.support_and_resistance(window_size_days=28, prominence=0.001, wlen=14, plateau_size=3)
)

In [7]:
dfs_temp.orderBy(["currency_pair","time"]).show()

+-------------------+-------+-------+-------+-------+------+-------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|               time|   open|   high|    low|  close|volume|currency_pair|         sma_1_day|         ema_1_day|    volatility_1_day|         sma_3_day|         ema_3_day|    volatility_3_day|
+-------------------+-------+-------+-------+-------+------+-------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|2019-01-07 15:00:00|0.71342|0.71349|0.71331|0.71336| 348.0|       AUDUSD|0.7133600115776062|              NULL|                NULL|0.7133600115776062|              NULL|                NULL|
|2019-01-07 16:00:00| 0.7134|0.71401|0.71318|0.71335|3762.0|       AUDUSD|0.7133550047874451|0.7133540034294129|3.540335274915098...|0.7133550047874451|0.7133540034294129|3.540335274915098...|
|2019-01-07 17:00:00|0.71345|0.7138